In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import csv
import json
option = webdriver.ChromeOptions()
#option.add_argument("--incognito")
#option.add_argument("--headless")
import undetected_chromedriver as uc
#option.add_argument("user-data-dir=C:\\Users\\PC2\\AppData\\Local\\Google\\Chrome\\User Data")#to open a window with your default profile (aka your google acc)

import random



In [2]:
 
def get_fixtures(driver, season_id):
    driver.get(f"https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/{season_id}/England-Premier-League")
    try:
        driver.maximize_window()
    except:
        pass
    try:
        driver.find_element(By.XPATH,"//span[contains(text(),'AGREE')]").click()
    except:
        pass
    time.sleep(3)
    fixtures = driver.find_element(By.XPATH, "//a[contains(text(),'Fixtures')]")
    fixtures.click()
    time.sleep(random.randint(2,6))
    return driver.page_source


def get_games(page_source):
    soup = BeautifulSoup(page_source, "lxml")
    games = soup.find_all("div", class_=lambda x: x in ("divtable-row col12-lg-12 col12-m-12 col12-s-12 col12-xs-12", "divtable-row col12-lg-12 col12-m-12 col12-s-12 col12-xs-12 alt"))
    return games


def get_game_stats(driver, game):
    game_stats = []
    try:
        result = [i.text for i in game.find("a", attrs={"class": "stacked-match-link result-1"}).find_all("div")]
        teams = [i.text for i in game.find_all("div", attrs={"class", "team"})][0:2]
    except:
        return None
    game_stats.extend(teams)
    game_stats.extend(result)
    match_report = game.find("a", attrs={"class": "match-link match-report rc"}).get('href')
    driver.get(f"https://1xbet.whoscored.com{match_report.replace('MatchReport', 'Live')}")
    time.sleep(random.randint(2,5))
    soup = BeautifulSoup(driver.page_source, "lxml")
    stats = soup.find_all("li", class_="has-stats")
    st = []
    for stat in stats[:8]:
        div = stat.find("div", attrs={"class": "match-centre-stat-values"})
        values = [i.text for i in div.find_all("span") if i.text != '-']
        st.append(values)
    driver.back()
    time.sleep(random.randint(2,5))
    game_stats.extend(st)
    return game_stats


def write_to_file(data,filename):

    with open(filename, 'w') as outfile: 
            json.dump(data, outfile,indent=0)
    
def read_from_file(filename):
    try:
        with open(filename, "r") as f:
            data = json.load(f)
    except FileNotFoundError:
        data = []
    return data

def get_season_data(season_id):
    columns = ["home_team","away_team","home_goals","away_goals","home_shots","away_shots","home_possesion","away_possesion","home_pass_success","away_pass_success","home_dribbles","away_dribbles","home_aerials_won","away_aerials_won","home_tackles","away_tackles","home_corners","away_corners","home_dispossessed","away_dispossessed"]
    driver = webdriver.Chrome(executable_path="C:\\Users\PC2\\Downloads\\chromedriver_win32 (4)\\chromedriver.exe")
    driver.implicitly_wait(30)
    
    # Read the data from the JSON file
    data = read_from_file(f"{season_id}.json")
    index  = len(data)

    source = get_fixtures(driver, season_id)
    for i in range(10):
        if i != 0:
            for y in range(i):
                driver.find_element(By.XPATH, "//div[contains(@id,'date-controller')]/a").click()
                time.sleep(3)
            source = driver.page_source
        games = get_games(source)
        
        for game in games:
            game_stats = get_game_stats(driver, game)
            if game_stats is not None:
                li = {}
                x = 0
                for s in game_stats:
                    if not isinstance(s,list):
                        li[columns[x]]=s
                        x+=1
                    else:
          
                        li[columns[x]]=s[0]
                        li[columns[x+1]]=s[1]
                        x+=2
                data.append(li)
                write_to_file(data,f"{season_id}.json")
        i+=len(games)
    driver.quit()


 
        
    

    
  

In [3]:

season_ids = [9075]
season=[2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
columns = ["home_team","away_team","home_goals","away_goals","home_shots","away_shots","home_possesion","away_possesion","home_pass_success","away_pass_success","home_dribbles","away_dribbles","home_aerials_won","away_aerials_won","home_tackles","away_tackles","home_corners","away_corners","home_dispossessed","away_dispossessed"]
for id in season_ids:
    get_season_data(id)
  





C:\Users\PC2\AppData\Local\Temp/ipykernel_23028/951456524.py:64: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:\\Users\PC2\\Downloads\\chromedriver_win32 (4)\\chromedriver.exe")


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.100)
Stacktrace:
Backtrace:
	(No symbol) [0x011237D3]
	(No symbol) [0x010B8B81]
	(No symbol) [0x00FBB36D]
	(No symbol) [0x00F9E5DA]
	(No symbol) [0x0100F0CB]
	(No symbol) [0x01020A56]
	(No symbol) [0x0100B216]
	(No symbol) [0x00FE0D97]
	(No symbol) [0x00FE253D]
	GetHandleVerifier [0x0139ABF2+2510930]
	GetHandleVerifier [0x013C8EC1+2700065]
	GetHandleVerifier [0x013CC86C+2714828]
	GetHandleVerifier [0x011D3480+645344]
	(No symbol) [0x010C0FD2]
	(No symbol) [0x010C6C68]
	(No symbol) [0x010C6D4B]
	(No symbol) [0x010D0D6B]
	BaseThreadInitThunk [0x75637D69+25]
	RtlInitializeExceptionChain [0x775FBB9B+107]
	RtlClearBits [0x775FBB1F+191]


In [ ]:
{}